# Zero-shot Named Entity Recognition (NER) - Model Comparison

This notebook compares different methods for named entity recognition by comparing their results on three benchmark datasets.

## Setup

Install required libraries and import necessary modules

### Install Dependencies

In [ ]:
!pip install datasets transformers scikit-learn gliner gliner-spacy
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

### Import Libraries

In [ ]:
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import spacy
from gliner_spacy.pipeline import GlinerSpacy
from gliner import GLiNER
import getpass
import os
import openai
import time

## Dataset Preparation - CoNLL-2003

### Load and Prepare Datasets

In [ ]:
max_datapoints = 1000

label_map_conll2003 = {
    "O": 0,
    "B-PERSON": 1,
    "I-PERSON": 2,
    "B-ORGANISATION": 3,
    "I-ORGANISATION": 4,
    "B-LOCATION": 5,
    "I-LOCATION": 6,
    "B-MISCELLANEOUS": 7,
    "I-MISCELLANEOUS": 8,
}

label_map_wnut2017 = {
    "O": 0,
    "B-CORPORATION": 1,
    "I-CORPORATION": 2,
    "B-CREATIVE-WORK": 3,
    "I-CREATIVE-WORK": 4,
    "B-GROUP": 5,
    "I-GROUP": 6,
    "B-LOCATION": 7,
    "I-LOCATION": 8,
    "B-PERSON": 9,
    "I-PERSON": 10,
    "B-PRODUCT": 11,
    "I-PRODUCT": 12
}

label_map_ontonotes5 = {
    "O": 0,
    "B-CARDINAL": 1,
    "B-DATE": 2,
    "I-DATE": 3,
    "B-PERSON": 4,
    "I-PERSON": 5,
    "B-NORP": 6,
    "B-GPE": 7,
    "I-GPE": 8,
    "B-LAW": 9,
    "I-LAW": 10,
    "B-ORGANISATION": 11,
    "I-ORGANISATION": 12,
    "B-PERCENT": 13,
    "I-PERCENT": 14,
    "B-ORDINAL": 15,
    "B-MONEY": 16,
    "I-MONEY": 17,
    "B-WORK_OF_ART": 18,
    "I-WORK_OF_ART": 19,
    "B-FACILITY": 20,
    "I-FACILITY": 32,
    "B-TIME": 21,
    "I-TIME": 29,
    "B-LOCATION": 23,
    "I-LOCATION": 27,
    "B-QUANTITY": 24,
    "I-QUANTITY": 25,
    "B-PRODUCT": 28,
    "I-PRODUCT": 34,
    "B-EVENT": 30,
    "I-EVENT": 31,
    "B-LANGUAGE": 33,
    "I-LANGUAGE": 36,
}


id_to_tag_conll2003 = {v: k for k, v in label_map_conll2003.items()}
id_to_tag_wnut2017 = {v: k for k, v in label_map_wnut2017.items()}
id_to_tag_ontonotes5 = {v: k for k, v in label_map_ontonotes5.items()}

fix_joined_tokens_map = {" .": ".", "( ": "(", " )": ")", " 's ": "'s ", "s ' ": "s' "}

In [ ]:
# CoNLL-2003 Dataset
def load_and_prepare_conll2003(max_datapoints=max_datapoints):
    dataset = load_dataset("conll2003", split="test", trust_remote_code=True)
    dataset = dataset.select(range(max_datapoints))

    sentence_column = []
    for datapoint in dataset:
        sentence = " ".join(datapoint["tokens"])
        for before, after in fix_joined_tokens_map.items():
            sentence = sentence.replace(before, after)
        sentence_column.append(sentence)
    dataset = dataset.add_column("sentence", sentence_column)
    return dataset

In [ ]:
# WNUT-2017
def load_and_prepare_wnut2017(max_datapoints=max_datapoints):
    dataset = load_dataset("wnut_17", split="test", trust_remote_code=True)
    dataset = dataset.select(range(max_datapoints))

    sentence_column = []
    for datapoint in dataset:
        sentence = " ".join(datapoint["tokens"])
        for before, after in fix_joined_tokens_map.items():
            sentence = sentence.replace(before, after)
        sentence_column.append(sentence)
    dataset = dataset.add_column("sentence", sentence_column)
    return dataset

In [ ]:
# OntoNotes-5
def load_and_prepare_ontonotes5(max_datapoints=max_datapoints):
    dataset = load_dataset("tner/ontonotes5", split="test", trust_remote_code=True)
    dataset = dataset.select(range(max_datapoints))

    sentence_column = []
    for datapoint in dataset:
        sentence = " ".join(datapoint["tokens"])
        for before, after in fix_joined_tokens_map.items():
            sentence = sentence.replace(before, after)
        sentence_column.append(sentence)
    dataset = dataset.add_column("sentence", sentence_column)
    dataset = dataset.add_column("id", range(len(dataset)))
    dataset = dataset.rename_column("tags", "ner_tags")
    return dataset

In [ ]:
conll2003 = load_and_prepare_conll2003()
wnut2017 = load_and_prepare_wnut2017()
ontonotes5 = load_and_prepare_ontonotes5()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.83k [00:00<?, ?B/s]

ontonotes5.py:   0%|          | 0.00/2.88k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/3.68M [00:00<?, ?B/s]

ontonotes5/validation/0000.parquet:   0%|          | 0.00/515k [00:00<?, ?B/s]

ontonotes5/test/0000.parquet:   0%|          | 0.00/517k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/59924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8528 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8262 [00:00<?, ? examples/s]

In [ ]:
print(wnut2017[:5])
print(conll2003[:5])
print(ontonotes5[:5])

{'id': ['0', '1', '2', '3', '4'], 'tokens': [['&', 'gt', ';', '*', 'The', 'soldier', 'was', 'killed', 'when', 'another', 'avalanche', 'hit', 'an', 'army', 'barracks', 'in', 'the', 'northern', 'area', 'of', 'Sonmarg', ',', 'said', 'a', 'military', 'spokesman', '.'], ['&', 'gt', ';', '*', 'Police', 'last', 'week', 'evacuated', '80', 'villagers', 'from', 'Waltengoo', 'Nar', 'where', 'dozens', 'were', 'killed', 'after', 'a', 'series', 'of', 'avalanches', 'hit', 'the', 'area', 'in', '2005', 'in', 'the', 'south', 'of', 'the', 'territory', '.'], ['&', 'gt', ';', '*', 'The', 'army', 'on', 'Thursday', 'recovered', 'the', 'bodies', 'of', 'ten', 'of', 'its', 'men', 'who', 'were', 'killed', 'in', 'an', 'avalanche', 'the', 'previous', 'day', '.'], ['&', 'gt', ';', '*', 'The', 'four', 'civilians', 'killed', 'included', 'two', 'children', 'of', 'a', 'family', 'whose', 'house', 'was', 'hit', 'by', 'a', 'separate', 'avalanche', ',', 'also', 'on', 'Wednesday', ',', 'a', 'police', 'spokesman', 'said', '.

In [ ]:
def extract_ner_tags(datasets):
    flattened_ner_tags = {}

    for i, dataset in enumerate(datasets):
        # Flatten ner_tags for the current dataset
        flattened_ner_tags = [
            tag for sentence_tags in dataset['ner_tags'] for tag in sentence_tags
        ]

    return flattened_ner_tags

In [ ]:
ner_tags_true_conll2003 = extract_ner_tags([conll2003])
ner_tags_true_wnut2017 = extract_ner_tags([wnut2017])
ner_tags_true_ontonotes5 = extract_ner_tags([ontonotes5])

In [ ]:
print(ner_tags_true_conll2003)
print(ner_tags_true_wnut2017)
print(ner_tags_true_ontonotes5)

[0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 5, 0, 5, 6, 6, 0, 5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 7, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 

### Data Alignment and Adjustment for Models

In [ ]:
def prepare_data(dataset, id_to_tag, model_type):
    if model_type == "gpt":
        prepared_data = []
        for datapoint in dataset:
            sentence = datapoint["sentence"]
            ner_tags = [(token, id_to_tag.get(tag, "O")) for token, tag in zip(datapoint["tokens"], datapoint["ner_tags"])]
            prepared_data.append({"sentence": sentence, "ner_tags": ner_tags})
    elif model_type == "bert":
        prepared_data = []
        for datapoint in dataset:
            ner_tags = [(token, id_to_tag.get(tag, "O")) for token, tag in zip(datapoint["tokens"], datapoint["ner_tags"])]
            prepared_data.append(ner_tags)
    elif model_type == "gliner":
        prepared_data = []
        for datapoint in dataset:
            sentence = datapoint["sentence"]
            tokens = datapoint["tokens"]
            true_labels = datapoint["ner_tags"]
            prepared_data.append({"sentence": sentence, "tokens": tokens, "true_labels": true_labels})
    else:
        raise ValueError(f"Invalid model_type: {model_type}. Choose from 'gpt', 'bert', or 'gliner'.")

    return prepared_data

#### GPT-4o-mini

In [ ]:
gpt_data_1 = prepare_data(conll2003, id_to_tag_conll2003, "gpt")
gpt_data_2 = prepare_data(wnut2017, id_to_tag_wnut2017, "gpt")
gpt_data_3 = prepare_data(ontonotes5, id_to_tag_ontonotes5, "gpt")

#### DistillBERT

In [ ]:
bert_data_1 = prepare_data(conll2003, id_to_tag_conll2003, "bert")
bert_data_2 = prepare_data(wnut2017, id_to_tag_wnut2017, "bert")
bert_data_3 = prepare_data(ontonotes5, id_to_tag_ontonotes5, "bert")

#### GliNER

In [ ]:
gliner_data_1 = prepare_data(conll2003, id_to_tag_conll2003, "gliner")
gliner_data_2 = prepare_data(wnut2017, id_to_tag_wnut2017, "gliner")
gliner_data_3 = prepare_data(ontonotes5, id_to_tag_ontonotes5, "gliner")

## Prepare the models

### GPT-4o-mini

In [ ]:
%pip install openai

In [ ]:
openai.api_key = "xxx"

In [ ]:
client = openai.OpenAI(api_key="xxx")

In [ ]:
def ask_openai(prompt: str, model: str = "gpt-4o-mini", max_tokens: int = 1024, temperature: float = 0.0) -> str:
    # Define the JSON schema
    json_schema = {
        "name": "named_entity_recognition",
        "schema": {
            "type": "object",
            "properties": {
                "ORGANISATION": {"type": "array", "items": {"type": "string"}},
                "LOCATION": {"type": "array", "items": {"type": "string"}},
                "PERSON": {"type": "array", "items": {"type": "string"}},
                "MISCELLANEOUS": {"type": "array", "items": {"type": "string"}},
                "CORPORATION": {"type": "array", "items": {"type": "string"}},
                "CREATIVE-WORK": {"type": "array", "items": {"type": "string"}},
                "GROUP": {"type": "array", "items": {"type": "string"}},
                "PRODUCT": {"type": "array", "items": {"type": "string"}},
                "CARDINAL": {"type": "array", "items": {"type": "string"}},
                "DATE": {"type": "array", "items": {"type": "string"}},
                "NORP": {"type": "array", "items": {"type": "string"}},
                "GPE": {"type": "array", "items": {"type": "string"}},
                "LAW": {"type": "array", "items": {"type": "string"}},
                "PERCENT": {"type": "array", "items": {"type": "string"}},
                "ORDINAL": {"type": "array", "items": {"type": "string"}},
                "MONEY": {"type": "array", "items": {"type": "string"}},
                "WORK_OF_ART": {"type": "array", "items": {"type": "string"}},
                "FACILITY": {"type": "array", "items": {"type": "string"}},
                "TIME": {"type": "array", "items": {"type": "string"}},
                "QUANTITY": {"type": "array", "items": {"type": "string"}},
                "EVENT": {"type": "array", "items": {"type": "string"}},
                "LANGUAGE": {"type": "array", "items": {"type": "string"}}
            },
            "additionalProperties": False
        }
    }

    # Make the API call
    response = openai.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a named entity recognition assistant. Extract entities and classify them into the specified categories."},
            {"role": "user", "content": prompt }
        ],
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        response_format={
            "type": "json_schema",
            "json_schema": json_schema
        }
    )

    # Return the response content
    return response.choices[0].message.content

#### Prompting

In [ ]:
gpt_config = {
    "conll2003": {
        "base_prompt": """Extract named entities from the given sentence and classify them into the following categories:
          •⁠  ORGANISATION: Organization named entity
          •⁠  LOCATION: Location named entity
          •⁠  PERSON: Person named entity
          •⁠  MISCELLANEOUS: Miscellaneous named entity

          Provide the output in JSON format with the keys "ORGANISATION", "LOCATION", "PERSON", and "MISCELLANEOUS". If a category has no entities, return an empty list for that key.

          Sentence: "sentence to be analyzed"
          """,

        "entity_types": ["PERSON", "ORGANISATION", "LOCATION", "MISCELLANEOUS"]
    },
    "wnut2017": {
        "base_prompt": """Extract named entities from the given sentence and classify them into the following categories:
          •⁠  LOCATION: Location named entity
          •⁠  PERSON: Person named entity
          •⁠  CORPORATION: Corporation named entity
          •⁠  CREATIVE-WORK: Creative-work named entity
          •⁠  GROUP: Group named entity
          •⁠  PRODUCT: Product named entity

          Provide the output in JSON format with the keys "LOCATION", "PERSON", "CORPORATION", "CREATIVE-WORK", "GROUP" and "PRODUCT". If a category has no entities, return an empty list for that key.

          Sentence: "sentence to be analyzed"
          """,
        "entity_types": ["LOCATION", "PERSON", "CORPORATION", "CREATIVE-WORK", "GROUP", "PRODUCT"]
    },
    "ontonotes5": {
        "base_prompt": """Extract named entities from the given sentence and classify them into the following categories:
          •⁠  CARDINAL: Cardinal number named entity
          •⁠  DATE: Date named entity
          •⁠  PERSON: Person named entity
          •⁠  NORP: Nationality, religious, or political group named entity
          •⁠  GPE: Geopolitical named entity
          •⁠  LAW: Law named entity
          •⁠  ORGANISATION: Organization named entity
          •⁠  PERCENT: Percent named entity
          •⁠  PRODUCT: Product named entity
          •⁠  ORDINAL: Ordinal number named entity
          •⁠  MONEY: Money named entity
          •⁠  WORK_OF_ART: Work of art named entity
          •⁠  FACILITY: Facility named entity
          •⁠  TIME: Time named entity
          •⁠  LOCATION: Location named entity
          •⁠  QUANTITY: Quantity named entity
          •⁠  EVENT: Event named entity
          •⁠  LANGUAGE: Language named entity

          Provide the output in JSON format with the keys "CARDINAL", "DATE", "PERSON", "NORP", "GPE", "LAW", "ORGANISATION", "PERCENT", "PRODUCT", "ORDINAL", "MONEY", "WORK_OF_ART", "FACILITY", "TIME", "LOCATION", "QUANTITY", "EVENT", "LANGUAGE". If a category has no entities, return an empty list for that key.

          Sentence: "sentence to be analyzed"
          """,
        "entity_types": ["CARDINAL", "DATE", "PERSON", "NORP", "GPE", "LAW", "ORGANISATION", "PERCENT", "PRODUCT", "ORDINAL", "MONEY", "WORK_OF_ART", "FACILITY", "TIME", "LOCATION", "QUANTITY", "EVENT", "LANGUAGE"]
    }
}


#### Predicting

In [ ]:
def predict_ner_gpt(dataset, config):
    gpt_predictions = []
    base_prompt = config["base_prompt"]
    entity_types = config["entity_types"]

    for datapoint in dataset:
        sentence = datapoint["sentence"]
        for attempt in range(1, 6):
          try:
              print(f"Attempt {attempt}", flush=True)
              answer = ask_openai(base_prompt + sentence)
              break
          except Exception as e:
              if attempt == 5: raise
              print(f"Failed: {e}, retrying in 5s...", flush=True)
              time.sleep(5)
        try:
            answer_json = json.loads(answer)
        except json.JSONDecodeError:
            print(f"Invalid JSON for sentence:\n{sentence}\nResponse:\n{answer}")
            answer_json = {entity_type: [] for entity_type in entity_types}
        gpt_predictions.append(answer_json)
        time.sleep(0.5)

    return gpt_predictions

In [ ]:
gpt_pred_1 = predict_ner_gpt(gpt_data_1, gpt_config["conll2003"])
print("\nGPT Predictions Example (Conll2003):\n", gpt_pred_1[:5])

In [ ]:
gpt_pred_2 = predict_ner_gpt(gpt_data_2, gpt_config["wnut2017"])
print("\nGPT Predictions Example (Wnut2017):\n", gpt_pred_2[:5])

In [ ]:
gpt_pred_3 = predict_ner_gpt(gpt_data_3, gpt_config["ontonotes5"])
print("\nGPT Predictions Example (OntoNotes5):\n", gpt_pred_3[:5])

In [ ]:
def transform_gpt_predictions(gpt_predictions):

    for i, json_dict in enumerate(gpt_predictions):
        for category in json_dict:
            if isinstance(json_dict[category], str):
                json_dict[category] = [json_dict[category]]
            elif isinstance(json_dict[category], list):
                continue
            else:
                print(f"Unknown value type '{type(json_dict[category])}' for category '{category}' with value '{json_dict[category]}'")
                json_dict[category] = []
    return gpt_predictions


# Apply the transformation to each dataset's predictions
gpt_pred_1_transformed = transform_gpt_predictions(gpt_pred_1)
gpt_pred_2_transformed = transform_gpt_predictions(gpt_pred_2)
gpt_pred_3_transformed = transform_gpt_predictions(gpt_pred_3)

# Example check for transformed predictions
print("\nTransformed GPT Predictions Example (Conll2003):\n", gpt_pred_1_transformed[:5])
print("\nTransformed GPT Predictions Example (Wnut2017):\n", gpt_pred_2_transformed[:5])
print("\nTransformed GPT Predictions Example (OntoNotes5):\n", gpt_pred_3_transformed[:5])



Transformed GPT Predictions Example (OntoNotes5):
 [{'CARDINAL': [], 'DATE': ['Friday'], 'PERSON': [], 'NORP': [], 'GPE': ['U.S.', 'non-U.S.'], 'LAW': [], 'ORGANISATION': ['Dow Jones Capital Markets Report'], 'PERCENT': [], 'PRODUCT': [], 'ORDINAL': [], 'MONEY': [], 'WORK_OF_ART': [], 'FACILITY': [], 'TIME': [], 'LOCATION': [], 'QUANTITY': [], 'EVENT': [], 'LANGUAGE': []}, {'CARDINAL': [], 'DATE': [], 'PERSON': [], 'NORP': [], 'GPE': [], 'LAW': [], 'ORGANISATION': ['Dow Chemical Co.'], 'PERCENT': [], 'PRODUCT': [], 'ORDINAL': [], 'MONEY': [], 'WORK_OF_ART': [], 'FACILITY': [], 'TIME': [], 'LOCATION': [], 'QUANTITY': [], 'EVENT': [], 'LANGUAGE': []}, {'CARDINAL': ['150', '8.55'], 'DATE': ['Oct. 15 , 2009'], 'PERSON': [], 'NORP': [], 'GPE': [], 'LAW': [], 'ORGANISATION': [], 'PERCENT': ['8.55 %'], 'PRODUCT': ['senior notes'], 'ORDINAL': [], 'MONEY': ['$ 150 million'], 'WORK_OF_ART': [], 'FACILITY': [], 'TIME': [], 'LOCATION': [], 'QUANTITY': [], 'EVENT': [], 'LANGUAGE': []}, {'CARDINAL'

#### Processing Results

In [ ]:
def process_ner_tags(dataset, gpt_transformed, label_map):

    gpt_pred_tags = []

    for i in range(len(dataset)):
        pred_entities_GPT = gpt_transformed[i]
        predicted_tags = [0] * len(dataset[i]['tokens'])

        for entity_type, entities in pred_entities_GPT.items():
            if entities is None:
                continue

            for entity in entities:
                try:
                    start_index = dataset[i]['sentence'].find(entity)
                    if start_index != -1:
                        end_index = start_index + len(entity)
                        token_indices = []

                        current_char_index = 0
                        for j, token in enumerate(dataset[i]['tokens']):
                            current_char_index = dataset[i]['sentence'].find(token, current_char_index)
                            if current_char_index != -1:
                                if start_index <= current_char_index < end_index:
                                    token_indices.append(j)
                                current_char_index += len(token)

                        if token_indices:
                            predicted_tags[token_indices[0]] = label_map.get(f"B-{entity_type.upper()}", 0)
                            for index in token_indices[1:]:
                                predicted_tags[index] = label_map.get(f"I-{entity_type.upper()}", 0)
                except ValueError:
                    print("Error: Entity not found")

        gpt_pred_tags.extend(predicted_tags)

    return gpt_pred_tags

In [ ]:
gpt_pred_conll2003 = process_ner_tags(conll2003, gpt_pred_1_transformed, label_map_conll2003)
gpt_pred_wnut2017 = process_ner_tags(wnut2017, gpt_pred_2_transformed, label_map_wnut2017)
gpt_pred_ontonotes5 = process_ner_tags(ontonotes5, gpt_pred_3_transformed, label_map_ontonotes5)


In [ ]:
print(gpt_pred_conll2003[:100])
print(gpt_pred_wnut2017[:100])
print(gpt_pred_ontonotes5[:100])

[7, 0, 5, 0, 0, 7, 0, 5, 0, 7, 7, 0, 1, 2, 5, 0, 5, 6, 6, 7, 5, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 7, 8, 0, 0, 0, 7, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 7, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 17, 17, 0, 13, 14, 28, 34, 0, 2, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


### DistillBERT

In [ ]:
label_map_distillbert = {
    "O": 0,
    "B-MISCELLANEOUS": 1,
    "I-MISCELLANEOUS": 2,
    "B-PERSON": 3,
    "I-PERSON": 4,
    "B-ORGANISATION": 5,
    "I-ORGANISATION": 6,
    "B-LOCATION": 7,
    "I-LOCATION": 8,
}

combined_label_map_distillbert = {
    "O": 0,
    "B-PERSON": 3,
    "I-PERSON": 4,
    "B-ORGANISATION": 5,
    "I-ORGANISATION": 6,
    "B-LOCATION": 7,
    "I-LOCATION": 8,
    "B-MISCELLANEOUS": 1,
    "I-MISCELLANEOUS": 2,
    "B-CORPORATION": 5,  # Same as ORGANISATION
    "I-CORPORATION": 6,  # Same as ORGANISATION
    "B-CREATIVE-WORK": 0,
    "I-CREATIVE-WORK": 0,
    "B-GROUP": 0,
    "I-GROUP": 0,
    "B-PRODUCT": 0,
    "I-PRODUCT": 0,
    "B-CARDINAL": 0,
    "B-DATE": 0,
    "I-DATE": 0,
    "B-NORP": 0,
    "B-GPE": 0,
    "I-GPE": 0,
    "B-LAW": 0,
    "I-LAW": 0,
    "B-PERCENT": 0,
    "I-PERCENT": 0,
    "B-ORDINAL": 0,
    "B-MONEY": 0,
    "I-MONEY": 0,
    "B-WORK_OF_ART": 0,
    "I-WORK_OF_ART": 0,
    "B-FACILITY": 0,
    "I-FACILITY": 0,
    "B-TIME": 0,
    "I-TIME": 0,
    "B-QUANTITY": 0,
    "I-QUANTITY": 0,
    "B-EVENT": 0,
    "I-EVENT": 0,
    "B-LANGUAGE": 0,
    "I-LANGUAGE": 0
}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(bert_data_1[:5])
print(bert_data_2[:5])
print(bert_data_3[:5])

[[('SOCCER', 'O'), ('-', 'O'), ('JAPAN', 'B-LOCATION'), ('GET', 'O'), ('LUCKY', 'O'), ('WIN', 'O'), (',', 'O'), ('CHINA', 'B-PERSON'), ('IN', 'O'), ('SURPRISE', 'O'), ('DEFEAT', 'O'), ('.', 'O')], [('Nadim', 'B-PERSON'), ('Ladki', 'I-PERSON')], [('AL-AIN', 'B-LOCATION'), (',', 'O'), ('United', 'B-LOCATION'), ('Arab', 'I-LOCATION'), ('Emirates', 'I-LOCATION'), ('1996-12-06', 'O')], [('Japan', 'B-LOCATION'), ('began', 'O'), ('the', 'O'), ('defence', 'O'), ('of', 'O'), ('their', 'O'), ('Asian', 'B-MISCELLANEOUS'), ('Cup', 'I-MISCELLANEOUS'), ('title', 'O'), ('with', 'O'), ('a', 'O'), ('lucky', 'O'), ('2-1', 'O'), ('win', 'O'), ('against', 'O'), ('Syria', 'B-LOCATION'), ('in', 'O'), ('a', 'O'), ('Group', 'O'), ('C', 'O'), ('championship', 'O'), ('match', 'O'), ('on', 'O'), ('Friday', 'O'), ('.', 'O')], [('But', 'O'), ('China', 'B-LOCATION'), ('saw', 'O'), ('their', 'O'), ('luck', 'O'), ('desert', 'O'), ('them', 'O'), ('in', 'O'), ('the', 'O'), ('second', 'O'), ('match', 'O'), ('of', 'O'), 

In [ ]:
def process_bert_data(bert_data, label_map_distillbert):
    sentences = [[word for word, label in sentence] for sentence in bert_data]
    labels = [[label for word, label in sentence] for sentence in bert_data]

    def tokenize_and_preserve_labels(sentence, labels):
        tokenized_inputs = tokenizer(sentence, truncation=True, is_split_into_words=True, return_tensors="pt")
        word_ids = tokenized_inputs.word_ids(batch_index=0)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_map_distillbert.get(labels[word_idx], 0))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        tokenized_inputs["labels"] = torch.tensor([label_ids])
        return tokenized_inputs

    return [
        tokenize_and_preserve_labels(sentence, lbl)
        for sentence, lbl in zip(sentences, labels)
    ]



In [ ]:
# Process each dataset
tokenized_inputs_1 = process_bert_data(bert_data_1, combined_label_map_distillbert)
tokenized_inputs_2 = process_bert_data(bert_data_2, combined_label_map_distillbert)
tokenized_inputs_3 = process_bert_data(bert_data_3, combined_label_map_distillbert)

#### Predicting

In [ ]:
model.eval()

bert_pred_tags_1, ner_tags_true_bert_1 = [], []
bert_pred_tags_2, ner_tags_true_bert_2 = [], []
bert_pred_tags_3, ner_tags_true_bert_3 = [], []

def get_predictions(model, tokenized_inputs, bert_pred_tags, ner_tags_true):
    with torch.no_grad():
        for tokenized_input in tokenized_inputs:
            input_ids = torch.nn.utils.rnn.pad_sequence(
                tokenized_input["input_ids"], batch_first=True, padding_value=0
            )
            attention_mask = torch.nn.utils.rnn.pad_sequence(
                tokenized_input["attention_mask"], batch_first=True, padding_value=0
            )
            labels = torch.nn.utils.rnn.pad_sequence(
                tokenized_input["labels"], batch_first=True, padding_value=-100
            )

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_ids = torch.argmax(logits, dim=2).numpy().flatten()
            label_ids = labels.numpy().flatten()

            bert_pred_tags.extend([pred_id for pred_id, lbl_id in zip(predicted_ids, label_ids) if lbl_id != -100])
            ner_tags_true.extend([lbl_id for lbl_id in label_ids if lbl_id != -100])

    assert len(ner_tags_true) == len(bert_pred_tags), f"Mismatch: {len(ner_tags_true)} true labels vs {len(bert_pred_tags)} predictions"


In [ ]:
# Get predictions for each dataset
get_predictions(model, tokenized_inputs_1, bert_pred_tags_1, ner_tags_true_bert_1)
get_predictions(model, tokenized_inputs_2, bert_pred_tags_2, ner_tags_true_bert_2)
get_predictions(model, tokenized_inputs_3, bert_pred_tags_3, ner_tags_true_bert_3)

In [ ]:
print("Dataset 1 - Predictions:", bert_pred_tags_1)
print("Dataset 1 - True Labels:", ner_tags_true_bert_1)
print("Dataset 2 - Predictions:", bert_pred_tags_2)
print("Dataset 2 - True Labels:", ner_tags_true_bert_2)
print("Dataset 3 - Predictions:", bert_pred_tags_3)
print("Dataset 3 - True Labels:", ner_tags_true_bert_3)

Dataset 1 - Predictions: [0, 0, 1, 0, 3, 0, 0, 5, 0, 0, 0, 0, 3, 4, 7, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 4, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,

### GliNER

In [ ]:
labels_gliner = {
    "conll2003": ["PERSON", "ORGANISATION", "LOCATION", "MISCELLANEOUS"],
    "wnut2017": ["LOCATION", "PERSON", "CORPORATION", "CREATIVE-WORK", "GROUP", "PRODUCT"],
    "ontonotes5": ["CARDINAL", "DATE", "PERSON", "NORP", "GPE", "LAW", "ORGANISATION", "PERCENT", "ORDINAL", "MONEY", "WORK_OF_ART", "FAC", "TIME", "LOCATION", "QUANTITY", "PRODUCT", "EVENT", "LANGUAGE"],
}

def create_gliner_pipeline(dataset_name):
    nlp = spacy.load("en_core_web_sm")
    labels = labels_gliner.get(dataset_name, [])

    if labels:
        nlp.add_pipe("gliner_spacy", config={"labels": labels})
    else:
        print(f"Warning: No labels defined for dataset '{dataset_name}'. GLiNER pipeline will not be added.")
    return nlp

In [ ]:
# For CoNLL-2003:
nlp_conll2003 = create_gliner_pipeline("conll2003")

# For WNUT-2017:
nlp_wnut2017 = create_gliner_pipeline("wnut2017")

# For OntoNotes-5:
nlp_ontonotes5 = create_gliner_pipeline("ontonotes5")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/792M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def make_ner_predictions(prepared_data, nlp):
    predicted_labels_nested = []
    for data in prepared_data:
        sentence = data["sentence"]
        tokens = data["tokens"]

        # Process sentence with SpaCy
        doc = nlp(sentence)
        spacy_tokens = [token.text for token in doc]

        # Initialize all tokens as "O"
        labels = ["O"] * len(tokens)

        # Map SpaCy's entities to BIO tags
        for ent in doc.ents:
            # Find start and end tokens for each entity
            entity_tokens = ent.text.split()
            start_index = next((i for i, token in enumerate(tokens) if tokens[i:i + len(entity_tokens)] == entity_tokens), -1)
            if start_index != -1:
                labels[start_index] = f"B-{ent.label_}"  # Beginning of entity
                for i in range(1, len(entity_tokens)):
                    labels[start_index + i] = f"I-{ent.label_}"  # Inside the entity

        predicted_labels_nested.append(labels)

    return predicted_labels_nested  # Return after the loop completes


In [ ]:
predicted_labels_nested_conll = make_ner_predictions(gliner_data_1, nlp_conll2003)
print(predicted_labels_nested_conll[:10])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[['O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O'], ['B-PERSON', 'I-PERSON'], ['B-LOCATION', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O'], ['B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'B-ORGANISATION', 'I-ORGANISATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'B-MISCELLANEOUS', 'I-MISCELLANEOUS', 'O', 'O', 'O', 'O', 'O'], ['O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORGANISATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O'], ['B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISCELLANEOUS', 'I-MISCELLANEOUS', 'O'], ['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-LOCATION', 'I-LOCATION', 'O', 'O', 'O', 'O', 'B-ORGANISATION

In [ ]:
predicted_labels_flat = [tag for sentence in predicted_labels_nested_conll for tag in sentence]
print(predicted_labels_flat[:50])

['O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'B-LOCATION', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'B-ORGANISATION', 'I-ORGANISATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'B-MISCELLANEOUS', 'I-MISCELLANEOUS', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'O']


In [ ]:
# test gpt function for label mapping
def map_labels_to_ids(predicted_labels_flat, label_mapping):
    return [label_mapping.get(label, 0) for label in predicted_labels_flat]

In [ ]:
# test function from above
mapped_predictions = map_labels_to_ids(predicted_labels_flat, label_map_conll2003)

# Output the result
print(mapped_predictions[:20])

[0, 0, 5, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 2, 5, 0, 5, 6, 6, 0]


In [ ]:
def predict_and_map_gliner(dataset, nlp_pipeline, label_map):
    predicted_labels_nested = make_ner_predictions(dataset, nlp_pipeline)  # Predict nested labels
    predicted_labels_flat = [tag for sentence in predicted_labels_nested for tag in sentence]  # Flatten the labels
    mapped_predictions = map_labels_to_ids(predicted_labels_flat, label_map)  # Map to numerical IDs
    return mapped_predictions

In [ ]:
# Example for conll2003:
gliner_predictions_conll2003 = predict_and_map_gliner(gliner_data_1, nlp_conll2003, label_map_conll2003)

# Example for wnut2017:
gliner_predictions_wnut2017 = predict_and_map_gliner(gliner_data_2, nlp_wnut2017, label_map_wnut2017)

# Example for ontonotes5:
gliner_predictions_ontonotes5 = predict_and_map_gliner(gliner_data_3, nlp_ontonotes5, label_map_ontonotes5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(gliner_predictions_conll2003[:20])
print(gliner_predictions_wnut2017[:20])
print(gliner_predictions_ontonotes5[:20])

[0, 0, 5, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 2, 5, 0, 5, 6, 6, 0]
[0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 23, 0, 23, 0, 0, 0, 0, 0, 0]


## Model Evaluation

In [ ]:
def evaluate_model_macro(ner_tags_true, predictions, dataset_name, label_map):
    from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

    reverse_label_map = {v: k for k, v in label_map.items()}

    true_labels = [reverse_label_map.get(tag, "UNKNOWN") for tag in ner_tags_true]
    pred_labels = [reverse_label_map.get(tag, "UNKNOWN") for tag in predictions]

    print(f"\n{dataset_name} Evaluation Results")
    print("Precision:", precision_score(true_labels, pred_labels, average="macro", zero_division=1))
    print("Recall:", recall_score(true_labels, pred_labels, average="macro", zero_division=1))
    print("F1-score:", f1_score(true_labels, pred_labels, average="macro", zero_division=1))
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, zero_division=1))

    return {
        "precision": precision_score(true_labels, pred_labels, average="macro", zero_division=1),
        "recall": recall_score(true_labels, pred_labels, average="macro", zero_division=1),
        "f1_score": f1_score(true_labels, pred_labels, average="macro", zero_division=1),
        "classification_report": classification_report(true_labels, pred_labels, zero_division=1, output_dict=True)
    }


In [ ]:
def evaluate_model_micro(ner_tags_true, predictions, dataset_name, label_map):
    from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

    reverse_label_map = {v: k for k, v in label_map.items()}

    true_labels = [reverse_label_map.get(tag, "UNKNOWN") for tag in ner_tags_true]
    pred_labels = [reverse_label_map.get(tag, "UNKNOWN") for tag in predictions]

    print(f"\n{dataset_name} Evaluation Results")
    print("Precision:", precision_score(true_labels, pred_labels, average="micro", zero_division=1))
    print("Recall:", recall_score(true_labels, pred_labels, average="micro", zero_division=1))
    print("F1-score:", f1_score(true_labels, pred_labels, average="micro", zero_division=1))
    print("\nClassification Report:")
    print(classification_report(true_labels, pred_labels, zero_division=1))

    return {
        "precision": precision_score(true_labels, pred_labels, average="micro", zero_division=1),
        "recall": recall_score(true_labels, pred_labels, average="micro", zero_division=1),
        "f1_score": f1_score(true_labels, pred_labels, average="micro", zero_division=1),
        "classification_report": classification_report(true_labels, pred_labels, zero_division=1, output_dict=True)
    }


In [ ]:
results = {
    "GPT-4o-mini": {
        "OntoNotes5": {
            "macro": evaluate_model_macro(ner_tags_true_ontonotes5, gpt_pred_ontonotes5, "OntoNotes5", label_map_ontonotes5),
            "weighted": evaluate_model_micro(ner_tags_true_ontonotes5, gpt_pred_ontonotes5, "OntoNotes5", label_map_ontonotes5)
        }
    },
  }


OntoNotes5 Evaluation Results
Precision: 0.4457668819531076
Recall: 0.5775928301888327
F1-score: 0.39725822189423915

Classification Report:
                precision    recall  f1-score   support

    B-CARDINAL       0.40      0.53      0.45       170
        B-DATE       0.86      0.32      0.47       421
       B-EVENT       0.06      1.00      0.12         2
    B-FACILITY       0.12      0.29      0.17        21
         B-GPE       0.80      0.72      0.76       369
    B-LANGUAGE       0.00      1.00      0.00         0
         B-LAW       0.07      0.67      0.12         6
    B-LOCATION       0.07      0.13      0.09        39
       B-MONEY       0.45      0.42      0.43       116
        B-NORP       0.68      0.69      0.68       148
     B-ORDINAL       0.20      0.45      0.28        20
B-ORGANISATION       0.80      0.73      0.76       645
     B-PERCENT       0.74      0.68      0.71       114
      B-PERSON       0.59      0.63      0.61       294
     B-PRODUCT   

In [ ]:
results = {
    "GPT-4o-mini": {
        "Conll2003": {
            "macro": evaluate_model_macro(ner_tags_true_conll2003, gpt_pred_conll2003, "Conll2003", label_map_conll2003),
            "weighted": evaluate_model_micro(ner_tags_true_conll2003, gpt_pred_conll2003, "Conll2003", label_map_conll2003)
        },
        "Wnut2017": {
            "macro": evaluate_model_macro(ner_tags_true_wnut2017, gpt_pred_wnut2017, "Wnut2017", label_map_wnut2017),
            "weighted": evaluate_model_micro(ner_tags_true_wnut2017, gpt_pred_wnut2017, "Wnut2017", label_map_wnut2017)
        },
        "OntoNotes5": {
            "macro": evaluate_model_macro(ner_tags_true_ontonotes5, gpt_pred_ontonotes5, "OntoNotes5", label_map_ontonotes5),
            "weighted": evaluate_model_micro(ner_tags_true_ontonotes5, gpt_pred_ontonotes5, "OntoNotes5", label_map_ontonotes5)
        }
    },
    "DistillBERT": {
        "Conll2003": {
            "macro": evaluate_model_macro(ner_tags_true_bert_1, bert_pred_tags_1, "Conll2003", label_map_distillbert),
            "weighted": evaluate_model_micro(ner_tags_true_bert_1, bert_pred_tags_1, "Conll2003", label_map_distillbert)
        },
        "Wnut2017": {
            "macro": evaluate_model_macro(ner_tags_true_bert_2, bert_pred_tags_2, "Wnut2017", label_map_distillbert),
            "weighted": evaluate_model_micro(ner_tags_true_bert_2, bert_pred_tags_2, "Wnut2017", label_map_distillbert)
        },
        "OntoNotes5": {
            "macro": evaluate_model_macro(ner_tags_true_bert_3, bert_pred_tags_3, "OntoNotes5", label_map_distillbert),
            "weighted": evaluate_model_micro(ner_tags_true_bert_3, bert_pred_tags_3, "OntoNotes5", label_map_distillbert)
        }
    },
    "GliNER": {
        "Conll2003": {
            "macro": evaluate_model_macro(ner_tags_true_conll2003, gliner_predictions_conll2003, "Conll2003", label_map_conll2003),
            "weighted": evaluate_model_micro(ner_tags_true_conll2003, gliner_predictions_conll2003, "Conll2003", label_map_conll2003)
        },
        "Wnut2017": {
            "macro": evaluate_model_macro(ner_tags_true_wnut2017, gliner_predictions_wnut2017, "Wnut2017", label_map_wnut2017),
            "weighted": evaluate_model_micro(ner_tags_true_wnut2017, gliner_predictions_wnut2017, "Wnut2017", label_map_wnut2017)
        },
        "OntoNotes5": {
            "macro": evaluate_model_macro(ner_tags_true_ontonotes5, gliner_predictions_ontonotes5, "OntoNotes5", label_map_ontonotes5),
            "weighted": evaluate_model_micro(ner_tags_true_ontonotes5, gliner_predictions_ontonotes5, "OntoNotes5", label_map_ontonotes5)
        }
    }
}

# Save results to JSON file
with open("evaluation_results_final.json", "w") as f:
    json.dump(results, f, indent=4)



Conll2003 Evaluation Results
Precision: 0.6190161494604857
Recall: 0.7282086445042979
F1-score: 0.6220660510742287

Classification Report:
                 precision    recall  f1-score   support

     B-LOCATION       0.57      0.92      0.71       446
B-MISCELLANEOUS       0.03      0.31      0.05       197
 B-ORGANISATION       0.78      0.47      0.58       523
       B-PERSON       0.96      0.94      0.95       672
     I-LOCATION       0.51      0.88      0.65        68
I-MISCELLANEOUS       0.07      0.55      0.12        83
 I-ORGANISATION       0.70      0.79      0.74       223
       I-PERSON       0.97      1.00      0.98       493
              O       0.98      0.70      0.82      9440

       accuracy                           0.72     12145
      macro avg       0.62      0.73      0.62     12145
   weighted avg       0.93      0.72      0.80     12145


Conll2003 Evaluation Results
Precision: 0.715850144092219
Recall: 0.715850144092219
F1-score: 0.715850144092219

Cl